# 誤差逆伝播法（Backpropagation）

書籍には加算レイヤ，乗算レイヤそれぞれ微分した内容が載っている．

微分自体はそれぞれのレイヤ（というよりノード）でできるから，それを逆方向に掛け算していけばよい（ほへぇって感じだった）
微分＝重みをどちら方向にどれだけ更新すればいいか，という指標なわけで，こいつが出力から入力まで一気に計算できると万々歳．

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [5]:
apple = 100
apple_num = 2
tax = 1.1

In [4]:
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [6]:
apple_price = mul_apple_layer.forward(apple, apple_num)

In [7]:
price = mul_tax_layer.forward(apple_price, tax)

In [8]:
price

220.00000000000003

In [9]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)

In [10]:
dapple_price, dtax

(1.1, 200)

In [11]:
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [12]:
dapple, dapple_num

(2.2, 110.00000000000001)

In [14]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### ReLUレイヤの実装
微分すると分かるけど，x>0の時のみ上流の値を下流に流すようになる．なるほど．
とは言え，これがどんなふうに活性化に繋がるのかは完全に理解できていない気がするので，ちょっと考える．

x>0のとき　→　上流では微分した値が0以上になっている　→　重みを微小だけ増加させると，ｘだけロスが増加する　→　要するに，重みを減らしたほうがいいですよ，ということを示している（と考えていいのやら）
となると，ReLUレイヤは重みを減らすだけ減らしていくための役割を持っているとも言えるわけだけれど，一方向に重みを更新することにどんなメリットが有るのかは良く分からん．過学習を防ぐことに繋がるのかな？そういわれるとしっくりくる気もする．

In [15]:
class Relu:
    """全ての関数について，numpy配列が利用されることが前提になっている"""
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [16]:
import numpy as np

In [18]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
x

array([[ 1. , -0.5],
       [-2. ,  3. ]])

In [20]:
mask = (x <= 0)
mask

array([[False,  True],
       [ True, False]], dtype=bool)

In [21]:
print(mask)

[[False  True]
 [ True False]]


### Sigmoidレイヤの実装
forwardに関してはお馴染みの数式をそのまんま実装するだけでいいが，逆に関しては一つ一つ考えて微分していく．詳細はP145あたりを参照すること．まぁ普通に微分できるのではと言われたらそれまで感はあるんだけれども，でも誤差逆伝播法ということを考える意味では，除算，乗算とひとつひとつ見ていかないと分からないと言うのはなるほど感ある．

In [24]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [25]:
X = np.random.rand(2)

In [26]:
X

array([ 0.04601527,  0.71486084])

In [27]:
W = np.random.rand(2,3)

In [28]:
W

array([[ 0.36599364,  0.19805854,  0.84081816],
       [ 0.41233778,  0.10931403,  0.20312359]])

In [29]:
z = np.random.rand(1,2)

In [30]:
z

array([[ 0.50892186,  0.23363901]])

In [31]:
B = np.random.rand(3,)

In [32]:
B

array([ 0.48309813,  0.21539705,  0.75695691])

In [33]:
Y = np.dot(X, W) + B

In [34]:
Y

array([ 0.79470355,  0.30265509,  0.94085248])